In [1]:
# Copyright(C) 2021 刘珅珅
# Environment: python 3.7
# Date: 2021.3.7
# 旅行商问题：lintcode 816

In [10]:
## 暴力DFS+剪枝
class Result:
    def __init__(self):
        self.min_cost = float('inf')
        
class Solution:
    """
    @param n: an integer,denote the number of cities
    @param roads: a list of three-tuples,denote the road between cities
    @return: return the minimum cost to travel all cities
    """
    def minCost(self, n, roads):
        # Write your code here
        
        ## 构建图，题目中的参数roads是一个序列，需要构建图，才好查找下一个城市
        graph = self.build_graph(n, roads)
        
        ## DFS搜索，题目需要从城市1开始，初始的visited中也要添加1进去
        result = Result()
        self.dfs(1, n, [1], set([1]), 0, graph, result)
        return result.min_cost
    
    ## 剪枝：剪去部分路径，并不是在剪枝中直接查找最优解
    ## 如果要把city加入到路径中，就判断添加了city的path是否有更优的情况，注意并不是判断它是否是最优解
    ## 这里的剪枝算法是，判断路径中两个连接点path[i-1]和path[i]的cost+path最后一个结点path[-1]到city的cost(如果city加入路径，path之前的最后结点与city相连)
    ## 是否大于从path[i-1]连接到path[-1]+path[i]连接到city，如果大于，说明目前的path中添加city肯定不是最优解
    ## 例如：path:[1,2,3,4,5,6,7]，city为8，剪枝算法中会判断graph[2][3]+graph[7][8] > graph[2][7] + graph[3][8]
    ## graph[3][4]+graph[7][8] > graph[3][7] + graph[4][8]...等情况，只有有1个满足，就说明8不适合进入path
    ## 这里只判断连接到path最后1个结点7的情形，因为它之前的结点在之前剪枝操作已判断过了
    ## 只选择两个相邻结点是为了简化剪枝运算，毕竟剪枝只是剪去部分路径
    def has_better_path(self, city, path, graph):
        for i in range(1, len(path)):  ## path的第1个值是城市1
            ## 这里需要判断某些路径是否为有效路径，因为建图的时候，根据给出的roads建图，某些路径可能不存在
            ## 如果不判断是否有效，可能会导致KeyError
            if self.is_valid_path(graph, path[i - 1], path[-1]) and self.is_valid_path(graph, path[i], city):
                if graph[path[i - 1]][path[i]] + graph[path[-1]][city] > graph[path[i - 1]][path[-1]] + graph[path[i]][city]:
                    return True
        return False

    
    ## 一个城市只能通过1次，所以需要visited
    def dfs(self, city, n, path, visited, cost, graph, result):
        if len(visited) == n:  ## 已访问了n个城市
            result.min_cost = min(result.min_cost, cost)
            return
        
        ## 下一个城市
        for next_city in graph[city]:
            if next_city in visited:
                continue
            
            if self.has_better_path(next_city, path, graph):
                continue
            
            ## 选择进入next_city
            visited.add(next_city)
            path.append(next_city)
            self.dfs(next_city, n, path, visited, cost + graph[city][next_city], graph, result)
            visited.remove(next_city)
            path.pop()
        
    
    def build_graph(self, n, roads):
        ## 互动视频中的建图方式有点问题，1的下一个城市也有1,2的下一个城市也有2，并且cost为'inf'，这样会导致某些输入样例会超时
        ## 还是如下建图方式比较好
        graph = {i: {} for i in range(1, n + 1)}
        for a, b, c in roads:
            if b not in graph[a]:
                graph[a][b] = c
            else:
                graph[a][b] = min(graph[a][b], c)
            if a not in graph[b]:
                graph[b][a] = c
            else:
                graph[b][a] = min(graph[b][a], c)
        return graph

    def is_valid_path(self, graph, prev, next):
        if next in graph[prev]:
            return True
        return False


In [11]:
solution = Solution()
n = 3
nums = [[1,2,1],[2,3,2],[1,3,3]]
n = 10
nums = [[1,2,2],[1,3,40],[1,4,43],[1,5,8],[1,6,38],[1,7,33],[1,8,24],[1,9,8],[1,10,5],[2,3,21],[2,4,48],[2,5,2],[2,6,42],[2,7,43],[2,8,19],[2,9,8],[2,10,15],[3,4,17],[3,5,4],[3,6,14],[3,7,8],[3,8,9],[3,9,46],[3,10,44],[4,5,11],[4,6,2],[4,7,49],[4,8,35],[4,9,17],[4,10,32],[5,6,44],[5,7,50],[5,8,20],[5,9,34],[5,10,20],[6,7,14],[6,8,23],[6,9,26],[6,10,35],[7,8,14],[7,9,2],[7,10,9],[8,9,24],[8,10,6],[9,10,25]]
print(solution.minCost(n, nums))

{1: {2: 2, 3: 40, 4: 43, 5: 8, 6: 38, 7: 33, 8: 24, 9: 8, 10: 5}, 2: {1: 2, 3: 21, 4: 48, 5: 2, 6: 42, 7: 43, 8: 19, 9: 8, 10: 15}, 3: {1: 40, 2: 21, 4: 17, 5: 4, 6: 14, 7: 8, 8: 9, 9: 46, 10: 44}, 4: {1: 43, 2: 48, 3: 17, 5: 11, 6: 2, 7: 49, 8: 35, 9: 17, 10: 32}, 5: {1: 8, 2: 2, 3: 4, 4: 11, 6: 44, 7: 50, 8: 20, 9: 34, 10: 20}, 6: {1: 38, 2: 42, 3: 14, 4: 2, 5: 44, 7: 14, 8: 23, 9: 26, 10: 35}, 7: {1: 33, 2: 43, 3: 8, 4: 49, 5: 50, 6: 14, 8: 14, 9: 2, 10: 9}, 8: {1: 24, 2: 19, 3: 9, 4: 35, 5: 20, 6: 23, 7: 14, 9: 24, 10: 6}, 9: {1: 8, 2: 8, 3: 46, 4: 17, 5: 34, 6: 26, 7: 2, 8: 24, 10: 25}, 10: {1: 5, 2: 15, 3: 44, 4: 32, 5: 20, 6: 35, 7: 9, 8: 6, 9: 25}}
52


In [12]:
n = 5
nums = [[1,2,9],[2,3,1],[3,4,9],[4,5,4],[2,4,3],[1,3,2],[5,4,9]]
print(solution.minCost(n, nums))

{1: {2: 9, 3: 2}, 2: {1: 9, 3: 1, 4: 3}, 3: {2: 1, 4: 9, 1: 2}, 4: {3: 9, 5: 4, 2: 3}, 5: {4: 4}}


KeyError: 4

In [1]:
n = 5
{i:{j : float('inf') for j in range(1, n + 1)} for i in range(1, n + 1)}

{1: {1: inf, 2: inf, 3: inf, 4: inf, 5: inf},
 2: {1: inf, 2: inf, 3: inf, 4: inf, 5: inf},
 3: {1: inf, 2: inf, 3: inf, 4: inf, 5: inf},
 4: {1: inf, 2: inf, 3: inf, 4: inf, 5: inf},
 5: {1: inf, 2: inf, 3: inf, 4: inf, 5: inf}}